# 10 minutes to pandas 실습

![img](https://w.namu.la/s/dc3751ffeceb57dce747772687e305ffbe76136a2f1efbb44ddbd71afb6511fcae43e26da7b82076e980ad5e10b2d8414a63f3e708dfe81adf7fffbd04f8bd057a2f3e1ff7cfc03f021c16fdaf4287a9)

해리포터 데이터셋을 이용하여 아래의 기능을 실습합니다.

1. EDA
2. Merge
3. Grouping
4. Reshaping
5. Time Series

Dataset: [harry-potter-dataset](https://www.kaggle.com/gulsahdemiryurek/harry-potter-dataset/version/5)

## 1. EDA
데이터를 탐색합니다.
* pd.read_csv()
* df.columns
* df.value_counts()

In [1]:
# 라이브러리를 import 합니다.
import pandas as pd
import numpy as np

In [2]:
# 해리포터 dataset을 불러옵니다
url = "https://raw.githubusercontent.com/sheon-j/pandas-study/main/datasets/harry_potter.csv"
df = pd.read_csv(url, sep=';')
df.head(3)

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,1,Harry James Potter,Male,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,Black,Bright green,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,31 July 1980,NaN
1,2,Ronald Bilius Weasley,Male,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,Red,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1 March 1980,NaN
2,3,Hermione Jean Granger,Female,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,Brown,Brown,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,"19 September, 1979",NaN


In [3]:
# df의 칼럼은 아래와 같이 구성되어있습니다
print(*list(df.columns), sep=", ")

Id, Name, Gender, Job, House, Wand, Patronus, Species, Blood status, Hair colour, Eye colour, Loyalty, Skills, Birth, Death


In [4]:
# 필요한 컬럼으로 DataFrame을 재구성 합니다.
df = df[df["Death"].isna()==True][["Name", "Gender", "Job", "House"]]
df.head(3)

,Name,Gender,Job,House
0,Harry James Potter,Male,Student,Gryffindor
1,Ronald Bilius Weasley,Male,Student,Gryffindor
2,Hermione Jean Granger,Female,Student,Gryffindor


In [5]:
# 기숙사(House)의 구성을 살펴봅니다
    # 호그와트 기숙사인 Gryffindor, Slytherin, Ravenclaw, Hufflepuff 에 대해서만 다루겠습니다.
df["House"].value_counts()

Gryffindor                      27
Slytherin                       21
Ravenclaw                       14
Hufflepuff                       8
Beauxbatons Academy of Magic     3
Durmstrang Institute             1
Name: House, dtype: int64

In [6]:
# Job의 구성을 살펴봅니다
    # 호그와트 학생의 데이터를 사용하겠습니다
df["Job"].value_counts()

Student                                                                                                 45
Advance Guard                                                                                            2
Auror                                                                                                    2
Professor of Divination                                                                                  2
Professor of Transfiguration | Head of Gryffindor                                                        1
Professor of Potions                                                                                     1
Professor of Muggle Studies                                                                              1
Professor of Herbology | Head of Hufflepuff House                                                        1
Professor of Arithmancyat Hogwarts                                                                       1
Employee in the Beast Division of the

In [7]:
# 기숙사별 학생 데이터를 구성합니다. 
    # gryf(그리핀도르), slyt(슬리데린), ravn(레번클로), huff(후플푸프)
gryf = df[(df["House"]=="Gryffindor")&(df["Job"]=="Student")].reset_index(drop=True)
slyt = df[(df["House"]=="Slytherin")&(df["Job"]=="Student")].reset_index(drop=True)
ravn = df[(df["House"]=="Ravenclaw")&(df["Job"]=="Student")].reset_index(drop=True)
huff = df[(df["House"]=="Hufflepuff")&(df["Job"]=="Student")].reset_index(drop=True)

## 2. Merge
데이터 병합을 실습합니다.
* pd.concat()
* pd.merge()
* df.append()

In [8]:
# Q1. 각 기숙사의 데이터가 합쳐진 DataFrame을 만드세요. (concat)
hogwart = pd.concat([gryf, slyt, ravn, huff]).reset_index(drop=True)
hogwart.tail(10)

,Name,Gender,Job,House
35,Anthony Goldstein,Male,Student,Ravenclaw
36,Penelope Clearwater,Female,Student,Ravenclaw
37,Roger Davies,Male,Student,Ravenclaw
38,Marcus Belby,Male,Student,Ravenclaw
39,Justin Finch-Fletchley,Male,Student,Hufflepuff
40,Zacharias Smith,Male,Student,Hufflepuff
41,Hannah Abbott,Female,Student,Hufflepuff
42,Ernest Macmillan,Male,Student,Hufflepuff
43,Susan Bones,Female,Student,Hufflepuff
44,Edward Remus Lupin,Male,Student,Hufflepuff


In [9]:
# 기숙사별 Gender 항목을 집계한 테이블을 준비하였습니다.
def gender_count(df, name):
    df_gender = df.groupby("Gender").count()
    df_gender = df_gender[["Name"]].reset_index()
    df_gender = df_gender.rename(columns={"Name": name})
    return df_gender

gryf_gender = gender_count(gryf, "Gryf")
slyt_gender = gender_count(slyt, "Slyt")
ravn_gender = gender_count(ravn, "Ravn")
huff_gender = gender_count(huff, "Huff")

gryf_gender

,Gender,Gryf
0,Female,8
1,Male,11


In [10]:
# Q2. 모든 _gender 테이블을 Join 하세요. (merge)
left = pd.merge(gryf_gender, slyt_gender, on="Gender")
right = pd.merge(ravn_gender, huff_gender, on="Gender")
hogwart_gender = pd.merge(left, right, on="Gender")
hogwart_gender

,Gender,Gryf,Slyt,Ravn,Huff
0,Female,8,2,5,2
1,Male,11,8,5,4


In [11]:
# Q3. 원하는 기숙사에 자신의 데이터를 추가해보세요. (호그와트 입학을 축하합니다!)
huff
huff = huff.append(
    {
        "Name": "Simon Jeong",
        "Gender": "Male",
        "Job": "Student", 
        "House": "Hufflepuff",
    }
    , ignore_index=True
)
huff

,Name,Gender,Job,House
0,Justin Finch-Fletchley,Male,Student,Hufflepuff
1,Zacharias Smith,Male,Student,Hufflepuff
2,Hannah Abbott,Female,Student,Hufflepuff
3,Ernest Macmillan,Male,Student,Hufflepuff
4,Susan Bones,Female,Student,Hufflepuff
5,Edward Remus Lupin,Male,Student,Hufflepuff
6,Simon Jeong,Male,Student,Hufflepuff


## 3. Grouping
데이터 그룹화를 실습합니다.
* df.groupby()

In [12]:
np.random.seed(132)

# 호그와트의 1학기 성적이 나왔습니다.
hogwart["Score_1"] = np.random.randint(50, 101, len(hogwart))
hogwart.head()

,Name,Gender,Job,House,Score_1
0,Harry James Potter,Male,Student,Gryffindor,83
1,Ronald Bilius Weasley,Male,Student,Gryffindor,67
2,Hermione Jean Granger,Female,Student,Gryffindor,100
3,Neville Longbottom,Male,Student,Gryffindor,92
4,George Weasley,Male,Student,Gryffindor,88


In [13]:
# Q4. 호그와트의 기숙사별(House) 성적 평균을 집계하세요.
# 후플푸프 1등!
hogwart.groupby("House").mean().round(1)

,Score_1
House,
Gryffindor,79.9
Hufflepuff,83.8
Ravenclaw,80.5
Slytherin,73.9


In [14]:
# Q5. 호그와트의 남녀별 성적 평균을 집계하세요.
hogwart.groupby("Gender").mean().round(1)

,Score_1
Gender,
Female,78.4
Male,79.8


In [15]:
# Q6. 호그와트의 기숙사별 남녀 평균 성적을 집계하세요.
hogwart.groupby(["House", "Gender"]).mean().round(1)

Score_1
House      Gender         
Gryffindor Female     76.5
           Male       82.5
Hufflepuff Female     73.5
           Male       89.0
Ravenclaw  Female     83.8
           Male       77.2
Slytherin  Female     77.0
           Male       73.1

## 4. Reshaping
데이터 변형을 실습합니다.
* df.stack()
* df.unstack()
* pd.pivot_table()

In [16]:
np.random.seed(134)

# 호그와트의 2학기 성적이 나왔습니다.
hogwart["Score_2"] = np.random.randint(50, 101, len(hogwart))
hogwart.head()

,Name,Gender,Job,House,Score_1,Score_2
0,Harry James Potter,Male,Student,Gryffindor,83,85
1,Ronald Bilius Weasley,Male,Student,Gryffindor,67,88
2,Hermione Jean Granger,Female,Student,Gryffindor,100,100
3,Neville Longbottom,Male,Student,Gryffindor,92,100
4,George Weasley,Male,Student,Gryffindor,88,70


In [17]:
# 위에서 사용한 기숙사별 남녀 평균 성적입니다.
hogwart_house_gender_mean = hogwart.groupby(["House", "Gender"]).mean().round(1)
hogwart_house_gender_mean

Score_1  Score_2
House      Gender                  
Gryffindor Female     76.5     77.2
           Male       82.5     78.0
Hufflepuff Female     73.5     66.5
           Male       89.0     74.5
Ravenclaw  Female     83.8     85.0
           Male       77.2     75.6
Slytherin  Female     77.0     70.5
           Male       73.1     66.4

In [18]:
# Q7. 남녀 평균 성적 DataFrame의 다층 컬럼을 단일층 칼럼으로 변환하세요.
stacked = hogwart_house_gender_mean.stack()
stacked

House       Gender         
Gryffindor  Female  Score_1    76.5
                    Score_2    77.2
            Male    Score_1    82.5
                    Score_2    78.0
Hufflepuff  Female  Score_1    73.5
                    Score_2    66.5
            Male    Score_1    89.0
                    Score_2    74.5
Ravenclaw   Female  Score_1    83.8
                    Score_2    85.0
            Male    Score_1    77.2
                    Score_2    75.6
Slytherin   Female  Score_1    77.0
                    Score_2    70.5
            Male    Score_1    73.1
                    Score_2    66.4
dtype: float64

In [19]:
# Q8. 단일층 DataFrame을 다시 다층으로 변환하세요.
stacked.unstack()

Score_1  Score_2
House      Gender                  
Gryffindor Female     76.5     77.2
           Male       82.5     78.0
Hufflepuff Female     73.5     66.5
           Male       89.0     74.5
Ravenclaw  Female     83.8     85.0
           Male       77.2     75.6
Slytherin  Female     77.0     70.5
           Male       73.1     66.4

In [20]:
# Q9. pivot_table 기능을 통해 통계 테이블을 만들어보세요.
pd.pivot_table(
    data=hogwart, 
    values=["Score_1", "Score_2"],
    index=["House"],
    aggfunc={"Score_1": [max, min, np.median],
             "Score_2": [max, min, np.median]} 
)

Score_1              Score_2             
               max median   min     max median   min
House                                               
Gryffindor   100.0   83.0  53.0   100.0   74.0  55.0
Hufflepuff    97.0   85.5  64.0    83.0   75.0  58.0
Ravenclaw    100.0   84.5  53.0   100.0   83.5  58.0
Slytherin     90.0   73.5  58.0    79.0   70.0  51.0

## 5. Time Series
데이터의 시계열 특성을 실습합니다.

In [21]:
# Q10. 호그와트 교내 편의점에 다음과 같은 매출 기록이 있을 때, 월별 매출을 구하세요.
np.random.seed(0)
hogwart_conv = pd.DataFrame({
    "date": pd.date_range("2021-1-1", periods=120, freq="D"), 
    "sales": np.random.randint(0, 100, 120)
})
hogwart_conv.head()

,date,sales
0,2021-01-01,44
1,2021-01-02,47
2,2021-01-03,64
3,2021-01-04,67
4,2021-01-05,67


In [22]:
hogwart_conv.groupby(hogwart_conv["date"].dt.month).sum()

,sales
date,
1,1762
2,1146
3,1397
4,1609
